In [136]:
# === Step 1: Import libraries === 
from sentence_transformers import SentenceTransformer, util 
import numpy as np
import pandas as pd
import os
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
import string
import torch

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vince\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [137]:
# === Step 2: Define competency framework (blocks) ===
block_path = os.path.join("..", "Data", "Competency_block.csv")
block_df = pd.read_csv(block_path)

block_df.head()

,job,competency
0,Data Analyst,resolution de problemes; sens du detail; commu...
1,Data Scientist,pensee analytique; resolution de problemes; in...
2,NLP Engineer,intuition linguistique; comprehension semantiq...
3,Fullstack Developer,resolution de problemes; pensee logique; trava...
4,UX/UI Designer,recherche utilisateur; wireframing; prototypag...


In [138]:
# Transform in dictionnary
block_dict = block_df.set_index('job').to_dict('index')

# Séparer les compétences en liste
block_dict = {job: val['competency'].split('; ') for job, val in block_dict.items()}

In [139]:
print(block_dict)

{'Data Analyst': ['resolution de problemes', 'sens du detail', 'communication', 'storytelling de donnees', 'travail en equipe', 'pensee critique', 'excel', 'bon niveau sql', 'visualisation de donnees', 'tableaux de bord', 'bon niveau python'], 'Data Scientist': ['pensee analytique', 'resolution de problemes', 'intuition des donnees', 'experimentation', 'collaboration', 'curiosite', 'tres bon niveau python', 'r', 'analyse statistique', 'modelisation predictive', 'ingenierie des fonctionnalites'], 'NLP Engineer': ['intuition linguistique', 'comprehension semantique', 'resolution de problemes', 'creativite', 'sens du detail', 'travail en equipe', 'bon a tres bon niveau python', 'tokenisation', 'transformers', 'word embeddings', "recherche d'information"], 'Fullstack Developer': ['resolution de problemes', 'pensee logique', 'travail en equipe', 'adaptabilite', 'communication', 'design thinking', 'bon niveau html', 'bon niveau css', 'tres bon niveau javascript', 'bon niveau react', 'node.js

In [140]:
# === Step 3: load the user input ===
id_path = os.path.join("..", "Data", "id.txt")
with open(id_path, "r") as f:
    last_id = int(f.read().strip())

user_input_path = os.path.join("..", "Data", "User_input", f"{last_id}_profile.csv")
user_input_df = pd.read_csv(user_input_path)

user_input_df.head()

,experiences,interests,qualities,python_level,sql_level,html_level,css_level,hadoop_level,cloud_level
0,J'ai travaillé sur la sécurisation de réseaux ...,"Je suis passionné par la cybersécurité, l'iden...","Rigueur, esprit analytique, curiosité, sens cr...",3,2,1,1,2,4


In [141]:
def niveau(input_niveau):
    if input_niveau == 1:
        return "tres mauvais"
    elif input_niveau == 2:
        return "mauvais"
    elif input_niveau == 3:
        return "moyen"
    elif input_niveau == 4:
        return "bon"
    elif input_niveau == 5:
        return "très bon"

skills_list = [
    f"python niveau {niveau(user_input_df['python_level'].iloc[0])}, sql niveau {niveau(user_input_df['sql_level'].iloc[0])}",
    f"html niveau {niveau(user_input_df['html_level'].iloc[0])}, css niveau {niveau(user_input_df['css_level'].iloc[0])}",
    f"hadoop niveau {niveau(user_input_df['hadoop_level'].iloc[0])}",
    f"{niveau(user_input_df['cloud_level'].iloc[0])} connaissance infrastructure cloud"
]

user_input_df = user_input_df.drop(columns=["python_level", "sql_level", "html_level", "css_level", "hadoop_level", "cloud_level"])

user_input_df.head()

,experiences,interests,qualities
0,J'ai travaillé sur la sécurisation de réseaux ...,"Je suis passionné par la cybersécurité, l'iden...","Rigueur, esprit analytique, curiosité, sens cr..."


In [142]:
# === Step 4: Cleaning the user input ===
column_cleaning = user_input_df.columns

#List of stop words
stop_words = set(stopwords.words('french'))

for col in column_cleaning:
    # To lower case
    user_input_df[col] = user_input_df[col].astype(str).str.lower()
    
    #Delete punctuation
    user_input_df[col] = user_input_df[col].apply(
        lambda x: re.sub(f"[{string.punctuation}]", " ", x)
    )
    
    #Delete stopwords
    user_input_df[col] = user_input_df[col].apply(
        lambda x: " ".join([word for word in x.split() if word not in stop_words])
    )
    
user_input_df.head()

,experiences,interests,qualities
0,travaillé sécurisation réseaux gestion accès p...,passionné cybersécurité identification mitigat...,rigueur esprit analytique curiosité sens criti...


In [143]:
# Transform user input in list
user_input = []

for col in column_cleaning:
    user_input.append(user_input_df[col].iloc[0])
    
user_input = user_input + skills_list

print(user_input)

['travaillé sécurisation réseaux gestion accès plusieurs projets académiques stages réalisé tests intrusion audits vulnérabilités mis place protocoles chiffrement pare feu également participé simulations réponse incidents analyse journaux sécurité maîtrise outils monitoring gestion incidents protection systèmes information', 'passionné cybersécurité identification mitigation risques protection systèmes information aime étudier nouvelles vulnérabilités analyser menaces émergentes concevoir stratégies défense différents environnements it', 'rigueur esprit analytique curiosité sens critique attention détails capacité travailler équipe communiquer clairement problèmes sécurité solutions méthodique persévérant résoudre problèmes complexes prévenir incidents', 'python niveau moyen, sql niveau mauvais', 'html niveau tres mauvais, css niveau tres mauvais', 'hadoop niveau mauvais', 'bon connaissance infrastructure cloud']


In [144]:
# === Step 4: Load SBERT model for embeddings === 
# tester plusieurs models
model = SentenceTransformer("multi-qa-mpnet-base-dot-v1")  # Model optimised for semantic analysis
 
# Encode user inputs 
user_embeddings = model.encode(user_input, convert_to_tensor=True)

In [145]:
# === Step 5: Calculate semantic similarity for each block === 
block_scores = {} 
 
for block, competencies in block_dict.items(): 
    # Encode competency block phrases 
    block_embeddings = model.encode(competencies, convert_to_tensor=True) 
     
    # Compare each user input to competencies using cosine similarity 
    similarities = util.cos_sim(user_embeddings, block_embeddings) 
     
    # Take max similarity per user input and average across inputs 
    max_similarities = [float(sim.max()) for sim in similarities]   
    block_score = np.mean(max_similarities) 
     
    block_scores[block] = block_score 

# Obtain top 3 job similarity
top_3_blocks = sorted(block_scores.items(), key=lambda x: x[1], reverse=True)[:3]

print(top_3_blocks)

[('Big Data Engineer', np.float64(0.5615851495947156)), ('Cybersecurity Analyst', np.float64(0.5611069798469543)), ('Data Analyst', np.float64(0.5497014778000968))]
